In [9]:
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt 
from matplotlib import patches
import sqlite3 as db
import os
import config as cng
from typing import Iterable, Callable, Sequence, List, Dict, Optional
from skimage import io

In [10]:
for var, val in cng.__dict__.items():
    if (val is not None) and not var.startswith('_'):
        print(f"{var} = {val}") 

pi = 3.141592653589793
SRC_CLTN = Originals
TRGT_CLTN = Copies
REF_CLTN = Reference
CAM_CLTN = Cameras
SPAWNBOX_OBJ = spawnbox
CAMERA_OBJ_CENTER = camera_C
CAMERA_OBJ_CENTER_TOP = camera_C_TOP
CAMERA_OBJ_LEFT = camera_L
CAMERA_OBJ_RIGHT = camera_R
ROT_MUS = [0, 0, -1.5707963267948966]
ROT_STDS = [6.283185307179586, 6.283185307179586, 6.283185307179586]
DEFAULT_BBOX_MODE = full
COMMIT_INTERVAL = 32
RAND_SCALE_MU = 1
RAND_SCALE_STD = 0.2
CLASS_DICT = {'haddock': 0, 'hake': 1, 'herring': 2, 'mackerel': 3, 'redgurnard': 4, 'whiting': 5}
DEFAULT_SPAWNRANGE = (1, 6)
DEFAULT_ALTER_COLOR = (0.2, 1, 0.2, 1)
DEFAULT_MIXSHADER_FAC = 0.3
GENERATED_DATA_DIR = generated_data
IMAGE_DIR = images
IMAGE_NAME = img
DEFAULT_FILEFORMAT = PNG
DEFAULT_FILEFORMAT_EXTENSION = .png
BBOX_DB_FILE = bboxes.db
METADATA_FILE = metadata.txt
BBOX_DB_IMGRNR = imgnr
BBOX_DB_CLASS = class_
BBOX_MODE_CPS = cps
BBOX_MODE_XYZ = xyz
BBOX_MODE_FULL = full
BBOX_MODE_STD = std
BBOX_DB_TABLE_XYZ = bboxes_xyz
BBOX_DB_TABLE_CPS = 

In [11]:
class BlenderData:
    def __init__(self, data_dir: str):
        self.data_dir = data_dir
        self.sqlite_file = os.path.join(data_dir, cng.BBOX_DB_FILE)
        self.image_dir = os.path.join(data_dir, cng.IMAGE_DIR)
        self.con = db.connect(f'file:{self.sqlite_file}?mode=ro', uri=True)
        
        with open(os.path.join(data_dir, cng.METADATA_FILE)) as f:
            self.num2name = eval(f.readline())
        
    def query(self, q: str) -> pd.DataFrame:
        return pd.read_sql_query(q, self.con)
    
    def get_image(self, imgnr: int, suffixes: Sequence[str]):
        imgs = []
        for suffix in suffixes:
            try:
                imgs.append(io.imread(os.path.join(
                    self.image_dir, f'{cng.IMAGE_NAME}{imgnr}{suffix}{cng.DEFAULT_FILEFORMAT_EXTENSION}')
                ))
            except FileNotFoundError as e:
                imgs.append(None)
        return imgs
    
    def plot_std_bboxes(self, imgnr: int, suffix: str, ax: Optional['Axes'] = None):
        
        if ax is None:
            fig, ax = plt.subplots(figsize=(7,5))
        else:
            ax = plt.gca()
        
        img = self.get_image(imgnr, [suffix])[0]
        img_h, img_w = img.shape[:-1]
        bboxes = self.query(
            f'SELECT class_, x, y, w, h FROM bboxes_std WHERE imgnr == {imgnr}'
        ).values
        
        ax.imshow(img)
        
        for class_, x, y, w, h in bboxes:
            topLeftCorner = (x*img_h, y*img_w)
            ax.add_patch(patches.Rectangle(topLeftCorner, w*img_w, h*img_h, facecolor='none', edgecolor='red', linewidth=2))
            ax.text(x=topLeftCorner[0], y=topLeftCorner[1], s=self.num2name[int(class_)], bbox=dict(facecolor='red', alpha=0.7))

In [12]:
data = BlenderData('objdet_std_data')
data.plot_std_bboxes(2, '')

OperationalError: unable to open database file